### **Default Predict model Demo**  
---

In [ ]:
pip install --upgrade pip setuptools==58.0.0 wheel

In [ ]:
pip install pandas feast ydata_profiling scikit-learn==0.24.2 mlxtend ipywidgets seldon-core boto3==1.35.99 shap cryptography==38.0.4 pyopenssl==22.0.0

In [ ]:
import pandas as pd
import ydata_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from math import log as log
import os
import mlflow

pd.options.mode.chained_assignment = None

# этот блок закомментирован так как используется только на kaggle
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
#PATH_to_file = '/kaggle/input/sf-dst-scoring/'

# # # этот блок закомментирован так как используется только локальной машине
from importlib import reload
print(os.listdir('./data'))
PATH_to_file = './data/'

In [ ]:
import utils_module28072020 as utils

In [ ]:
RANDOM_SEED = 42
!pip freeze > requirements.txt
CURRENT_DATE = pd.to_datetime('11/08/2020')

## 2. Импорт данных

In [ ]:
df_train = pd.read_csv(PATH_to_file+'train.csv')
df_test = pd.read_csv(PATH_to_file+'test.csv')
pd.set_option('display.max_columns', None)
print('Размерность тренировочного датасета: ', df_train.shape)
display(df_train.head(2))
print('Размерность тестового датасета: ', df_test.shape)
display(df_test.head(2))

In [ ]:
# ВАЖНО! для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['Train'] = 1 # помечаем где у нас трейн
df_test['Train'] = 0 # помечаем где у нас тест

df = df_train.append(df_test, sort=False).reset_index(drop=True) # объединяем
#!Обратите внимание объединение датасетов является потенциальной опасностью для даталиков

In [ ]:
 # временной ряд (1)
time_cols = ['app_date']
# бинарные переменные (default не включаем в список) (5+1 = 6)
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']
# категориальные переменные (Train не включаем в список, так как мы сами его добавили) (3+1=4)
cat_cols = ['education', 'region_rating', 'home_address', 'work_address', 'sna', 'first_time']
# числовые переменные, client_id исключили из списка (8)
num_cols = ['age','decline_app_cnt','score_bki','bki_request_cnt','income','days']
# client_id не включаем в списки

### Приведение признаков к целевому виду

In [ ]:
df['age'] = np.log(df['age'] + 1)
df['decline_app_cnt'] = np.log(df['decline_app_cnt'] + 1)
df['bki_request_cnt'] = np.log(df['bki_request_cnt'] + 1)
df['income'] = np.log(df['income'] + 1)
df['education'] = df['education'].fillna('SCH')
df.app_date = pd.to_datetime(df.app_date, format='%d%b%Y')

start = df.app_date.min()
end = df.app_date.max()
df['days'] = (df.app_date - start).dt.days.astype('int')

In [ ]:
df

---
### Оценка корреляций

In [ ]:
utils.simple_heatmap('Матрица корреляции тренировочного датасета на числовых переменных',df[df['Train']==1], num_cols+['default'], 1.1, 1, 9)

***Резюме*** - сильно скорелированных между собой признаков нет, все берем в работу

### Значимость непрерывных переменных по ANOVA F test

In [ ]:
temp_df = df[df['Train']==1]
imp_num = pd.Series(f_classif(temp_df[num_cols], temp_df['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh', title='Значимость непрерывных переменных по ANOVA F test')

***Резюме*** - оценка плательщика БКИ (score_bki) самый значимый показатель по ANOVA F test, потом кол-во отказанных заявок (declain_app_cnt) и в конце возраст (age)

### Посмотрим на значимость категориальных и бинарных переменных 

In [ ]:
label_encoder = LabelEncoder()
df['education_l'] = label_encoder.fit_transform(df['education'])

# паралельно подготовим бинарные переменные и переведем их в числовой формат
# для бинарных признаков мы будем использовать LabelEncoder
label_encoder = LabelEncoder()
for column in bin_cols:
    df[column] = label_encoder.fit_transform(df[column])
    
# тут могут быть потенциальные даталики, но мы пока не придумали как это обработать,
# потому что далее по этим меткам формируются новые фичи по get_dummies

all_cat_and_bin_cols = cat_cols+bin_cols
all_cat_and_bin_cols.remove('education')
all_cat_and_bin_cols.append('education_l')
print(all_cat_and_bin_cols)

temp_df = df[df['Train']==1]
imp_cat = pd.Series(mutual_info_classif(temp_df[all_cat_and_bin_cols], temp_df['default'], discrete_features =True), index = all_cat_and_bin_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh', title = 'Значимость бин. и категор. переменных по Mutual information test')

***Резюме*** - Самым значимым признаком по Mutual information тесту является связь заемщика с клиентами банка (sna) и давность наличия информации о заемщике (first_time), потом идет рейтинг региона (region_rating) и в конце пол (sex).

## 3. Подготовка данных к машинному обучению
---

### Категориальные признаки

In [ ]:
# реализуем метод OneHotLabels через get_dummies
df=pd.get_dummies(df, prefix=cat_cols, columns=cat_cols)

### Стандартизация

In [ ]:
# стандартизацию проводим отдельно для трейна и теста, чтобы не допустить даталиков
utils.StandardScaler_df_and_filna_0(df[df['Train']==1], num_cols)

utils.StandardScaler_df_and_filna_0(df[df['Train']==0], num_cols)

### Удаление нечисловых критериев

In [ ]:
df.drop(['app_date', 'education_l'], axis=1, inplace=True)

## 4. Построение модели
---
### Разбиваем датасет на тренировочный и тестовый

In [ ]:
train_data = df.query('Train == 1').drop(['Train', 'client_id'], axis=1)
test_data = df.query('Train == 0').drop(['Train', 'client_id'], axis=1)

y = train_data.default.values            # наш таргет
X = train_data.drop(['default'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

### Создаём эксперимент в MLFLow

> Перед запуском следующего блока ячеек настройте переменные окружения для аутентификации и подключения к MLflow: 
> + MLFLOW_TRACKING_URI – адрес MLflow, работающего внутри проекта.
> + MLFLOW_TRACKING_TOKEN – токен аутентификации, который подтверждает права пользователя. Токен можно получить через Developer Tools браузера в заголовках запросов.

In [ ]:
import os
os.environ["MLFLOW_TRACKING_URI"] = 'https://{{Базовый URL платформы}}/project-api/{ID проекта}/mlflow/'
os.environ["MLFLOW_TRACKING_TOKEN"] = '{{Пользовательский Bearer токен}}'

In [ ]:
mlflow.set_experiment('autotestPredict') 

### Обучаем модель, генерируем результат и сравниваем с тестом

In [ ]:
model = LogisticRegression(random_state=RANDOM_SEED, 
                           C=1, 
                           class_weight= 'balanced', 
                           dual= False, 
                           fit_intercept= True, 
                           intercept_scaling= 1, 
                           l1_ratio= None, 
                           multi_class= 'auto', 
                           n_jobs= None, 
                           penalty= 'l2', 
                           solver = 'sag', 
                           verbose= 0, 
                           warm_start= False)

model.fit(X_train, y_train)

y_pred_prob = model.predict_proba(X_test)[:,1]
y_pred = model.predict(X_test)

### Оценка качества модели
---
### Качественные метрики

In [ ]:
# в первый раз инициируем глобальную переменную с предыдущим скором
utils.last_pred = np.zeros((3,len(y_test)))

In [ ]:
utils.test_last_pred(y_test, y_pred, y_pred_prob) if (utils.last_pred[0].max() == 0) else 0
utils.all_metrics(y_test, y_pred, y_pred_prob)

### Матрица ошибок

In [ ]:
utils.confusion_matrix_f(['Дефолтный','Не дефолтный'], y_test, y_pred, 1.2, normalize=False)

### ROC кривая

In [ ]:
utils.ROC_curve_with_area(y_test, y_pred_prob, 1.1)

***Резюме*** - прекрасный пример несостоятельности метрики ROC-AUC на не сбалансированных данных. Мы абсолютно не угадали дефолтных клиентов, тем самым обеспечили себе огромную ошибку второго рода и как следствие колосальный убыток, но ROC-AUC у нас высокий. Благо f1 как-то сигнализирует о том что что-то не впорядке. Надо посмотреть на метрику которая может оценивать эффективность алгоритма на несбалансированных данных - PRC-AUC.

### Precision-Recall кривая

In [ ]:
utils.PR_curve_with_area(y_test, y_pred, 1.1)

### Сохранение метрик, артефактов и графиков в MLFLow

> Перед запуском следующего блока ячеек настройте переменные окружения для аутентификации и подключения к MLflow: 
> + MLFLOW_TRACKING_URI – адрес MLflow, работающего внутри проекта.
> + MLFLOW_TRACKING_TOKEN – токен аутентификации, который подтверждает права пользователя. Токен можно получить через Developer Tools браузера в заголовках запросов.

In [ ]:
import os
os.environ["MLFLOW_TRACKING_URI"] = 'https://{{Базовый URL платформы}}/project-api/{ID проекта}/mlflow/'
os.environ["MLFLOW_TRACKING_TOKEN"] = '{{Пользовательский Bearer токен}}'

In [ ]:
from sys import version_info
import feast
import sklearn
import numpy
import dill
import joblib

conda_env={
    'channels': ['defaults'],
    'dependencies': [
      'python=3.8.10',
      'pip>=22.0, <24.0',
      'setuptools>=58.0, <72.0',
      {
        'pip': [
          'mlflow=={}'.format(mlflow.__version__),
          'numpy=={}'.format(numpy.__version__),
          'scikit-learn=={}'.format(sklearn.__version__),
          'joblib=={}'.format(joblib.__version__),
          'dill=={}'.format(dill.__version__),
        ],
      },
    ],
    'name': 'demo_env'
}
eval_data = X_test.copy()
eval_data["target"] = y_test

with mlflow.start_run() as run:
   model_info = mlflow.sklearn.log_model(model, "model", conda_env=conda_env)
   mlflow.set_tag("mlflow.source.type", "JOB")
   mlflow.set_tag("mlflow.source.name", "https://neogit.neoflex.ru/neoflex-mlops-center/demo-examples/demo")
   mlflow.set_tag("mlflow.source.git.commit", "c9a90ee")
   mlflow.set_tag("mlflow.source.git.branch", "main")
   mlflow.set_tag("mlflow.source.git.repoURL", "https://neogit.neoflex.ru/neoflex-mlops-center/demo-examples/demo")
   result = mlflow.evaluate(
       model_info.model_uri,
       eval_data,
       targets="target",
       model_type="classifier",
       dataset_name="default-predict",
       evaluators="default",
       evaluator_config={"explainability_nsamples": 1000},
   )